In [3]:
import sys
from pathlib import Path
sys.path.append(str(Path("..").resolve()))


In [4]:
import requests
import pandas as pd

import networkx as nx
from sage.all import *
from objects import *
from utils import gpn


### Fig generator for each graphs class

In [ ]:
output_dir = Path.cwd().parent.parent / "docs/figs"
output_dir.mkdir(parents=True, exist_ok=True)
max_nodes = 10


In [ ]:
for n in range(1, 9):
    try:
        g = BipartiteGraphs(n)
    except RuntimeError:
        continue

    for i, graph in enumerate(g.graphs, start=0):
        file_path = output_dir / "bipartite" / f"bipartite_{n}_nodes_{i}.png"
        g.plot(index=i, filename=str(file_path))


In [ ]:
for n in range(4, max_nodes + 1, 2):
    try:
        g = CubicGraphs(n)
    except RuntimeError:
        continue

    for i, graph in enumerate(g.graphs, start=0):
        file_path = output_dir / "cubic" / f"cubic_{n}_nodes_{i}.png"
        g.plot(index=i, filename=str(file_path))


In [ ]:
for n in range(1, 8):
    try:
        g = TriangleFreeGraphs(n)
    except RuntimeError:
        continue

    for i, graph in enumerate(g.graphs, start=1):
        deg_seq = g.degree_sequences[i-1]
        deg_str = "_".join(map(str, deg_seq))
        file_name = f"tf_{deg_str}.png"

        file_path = output_dir / "triangle-free" / file_name
        g.plot(index=i-1, filename=str(file_path))


Graph specifi class data generator

In [1]:
max_nodes = 10


In [ ]:
df_data = []

for n in range(1, max_nodes + 1):
    try:
        bg = BipartiteGraphs(n)
    except RuntimeError:
        continue
    for i, graph in enumerate(bg.graphs, start=1):
    
        df_data.append(
            [
                nx.to_graph6_bytes(graph, header=False).decode('utf-8').replace('\n', '').replace('\r', ''),
                "bipartite",
                graph.number_of_nodes(),
                graph.number_of_edges(),
                gpn(graph),
            ]
        )

for n in range(4, max_nodes + 1, 2):
    try:
        cg = CubicGraphs(n)
    except RuntimeError:
        continue

    for i, graph in enumerate(cg.graphs, start=1):

        df_data.append(
            [
                nx.to_graph6_bytes(graph, header=False).decode('utf-8').replace('\n', '').replace('\r', ''),
                "cubic",
                graph.number_of_nodes(),
                graph.number_of_edges(),
                gpn(graph),
            ]
        )

for n in range(1, max_nodes + 1):
    try:
        tg = TriangleFreeGraphs(n)
    except RuntimeError:
        continue

    for i, graph in enumerate(tg.graphs, start=1):

        df_data.append(
            [
                nx.to_graph6_bytes(graph, header=False).decode('utf-8').replace('\n', '').replace('\r', ''),
                "triangle-free",
                graph.number_of_nodes(),
                graph.number_of_edges(),
                gpn(graph),
            ]
        )

df = pd.DataFrame(
    df_data,
    columns=[ "graph6_encoding", "type", "num_nodes", "num_edges", "gpn_num"],
)

df.to_csv("../../data/generated/gpn_class_data.csv", index=False, encoding='utf-8')



All graph dataset construction from two perspectives:
- number of edges
- number of nodes


In [ ]:
# number of nodes generator
df_data = []
for n in range(1, 10):
    print(n)
    for g in graphs.nauty_geng(f"{n} -c"):
        g = g.networkx_graph()

        if not nx.is_connected(g):
            print("g is not connected")
        
        graph6_encoding = nx.to_graph6_bytes(g, header=False).decode('utf-8').replace('\n', '').replace('\r', '')
        num_nodes = g.number_of_nodes()
        num_edges = g.number_of_edges()
        gpn_value = gpn(g)

        df_data.append(
            [
                graph6_encoding,
                num_nodes,
                num_edges,
                gpn_value
            ]
        )

columns = [
    "graph6_encoding",
    "num_nodes",
    "num_edges",
    "gpn_num",
]

df = pd.DataFrame(df_data, columns=columns)
df.to_csv("../../data/generated/all_graphs_9n.csv", index=False)


### LOAD GE-data files (graph6 files)
(required for edge based generator)

In [ ]:
for i in range(1, 16):
    url = f"https://users.cecs.anu.edu.au/~bdm/data/ge{i}c.g6"
    local_filename = Path("../../data/loaded/ge_files/") / f"edge-based-ge{i}c.g6"
    
    print(f"Downloading {url} ...")
    
    try:
        resp = requests.get(url)
        resp.raise_for_status()
        
        with open(local_filename, "wb") as f:
            f.write(resp.content)
        
        print(f"saved as {local_filename}")
        
    except requests.RequestException as e:
        print(f"{url}: {e}")


In [ ]:
# num of edges based generator
df_data = []

for i in range(1, 15):
    print(f"loading {i}")
    local_file = Path("../../data/loaded/ge_files/") / f"edge-based-ge{i}c.g6"

    with open(local_file, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            g = nx.from_graph6_bytes(line.encode())
            df_data.append([g.number_of_nodes(), g.number_of_edges(), gpn(g)])

columns = ["num_nodes", "num_edges", "gpn_num"]
df = pd.DataFrame(df_data, columns=columns)
df.to_csv("../../data/generated/all_graphs_edge_data.csv", index=False)
